In [4]:
import tkinter as tk
import math
import OpenOPC
import pywintypes
import time
import threading
import functools
pywintypes.datetime = pywintypes.TimeType

In [41]:
class EStop(tk.Tk):
    def __init__(self, *args, **kwargs):
        tk.Tk.__init__(self,  *args, **kwargs)  
        self.opc = OpenOPC.client()
        self.opc.connect('Matrikon.OPC.Simulation.1')
        self.opc.write(('output.Q', True))
        self.opc.write(('output.Q_DELAY', True))
        self.parent = None
        self.geometry('500x500')
        self.buttonState = 1 
        self.ack = tk.IntVar()
        self.ack_nec = tk.IntVar()
        self.ackPrevious = 1
        self.delay = tk.StringVar()
        self.category = 0
        self.Q = 0
        self.Q_DELAY = 0
        self.ACK_REQ = 0
        self.DIAG = 0
        self.isRaisingEdge = 0
        
        self.displayFrame = tk.Frame(self)
        self.displayFrame['height'] = 125
        self.displayFrame['width'] = 400
        self.displayFrame.pack()
        self.displayFrame.place(x=50, y=320)
        self.displayFrame['bg'] = 'yellow'
        self.canvas = tk.Canvas(self.displayFrame)
        self.canvas.pack()
        self.canvas['width'] = 400
        self.canvas['height'] = 125
        self.LEDs = dict()
        self.LEDs['Q'] = (50, 10, 120, 80)
        self.LEDs['Q_Delay'] = (170, 10, 240, 80)
        self.LEDs['ACK_REQ'] = (290, 10, 360, 80) 
        self.canvas.create_oval(self.LEDs['Q'], outline="#000000", fill="#81eb8d", width=2)
        self.canvas.create_text(86, 90, font="Times 12 bold", text="Q")
        self.canvas.create_oval(self.LEDs['Q_Delay'], outline="#000000", fill="#81eb8d", width=2)
        self.canvas.create_text(203, 90, font="Times 12 bold", text="Q_Delay")
        self.canvas.create_oval(self.LEDs['ACK_REQ'], outline="#000000", fill="#f54e42", width=2)
        self.canvas.create_text(323, 90, font="Times 12 bold", text="ACK_REQ")
        
        # E-Stop Button
        self.eStopButton = tk.Button(self, text='E-STOP', fg='black', bg='red', command=self.eStopPressed, height=4, width=10)
        self.eStopButton.pack()
        self.eStopButton.place(x=320, y=200)
        
        #Acknowledegment
        self.ackButton = tk.Checkbutton(self, text='Acknowledgement', padx=20, variable=self.ack, onvalue=1, offvalue=0)
        self.ackButton.place(x=50, y = 150)
        self.ack.set(0)
        self.ackButton.config(font=("Times", 12, 'bold'))
        
        #Acknowledgement Necessary
        self.ackNecButton = tk.Checkbutton(self, text='Acknowledgement Necessary', padx=20, variable=self.ack_nec, onvalue=1, offvalue=0)
        self.ackNecButton.place(x=50, y = 200)
        self.ack_nec.set(1)
        self.ackNecButton.config(font=("Times", 12, 'bold'))
        
        #Time Delay
        self.timeDelayLabel = tk.Label(self, text="Time Delay", font=("Times", 12, 'bold'))
        self.timeDelayLabel.pack()
        self.timeDelayLabel.place(x=90, y=250)
        self.timeDelay = tk.Entry(self, font=("Times", 12, 'bold'), width = "5", textvariable=self.delay)
        self.delay.set('0')
        self.timeDelay.pack()
        self.timeDelay.place(x=190, y=250)
        
    def setLED(self, led):
        self.canvas.create_oval(self.LEDs[led], outline="#000000", fill="#81eb8d", width=2)
    def resetLED(self, led):
        self.canvas.create_oval(self.LEDs[led], outline="#000000", fill="#f54e42", width=2)
    
    def delayedStop(self):
        self.opc.write(('output.Q_DELAY', False))
        self.resetLED('Q_Delay')
        self.eStopButton['state'] = 'normal'
        self.eStopButton['text'] = 'Release'
        self.eStopButton['bg'] = 'green'
        
    def detectRaisingEdgeThread(self):
        while True:
            current = self.ack.get()
            if current == 1 and self.ackPrevious == 0:
                self.isRaisingEdge = 1
            self.ackPrevious = current
        
    def updateAckReq(self):
        ackNecStatus = self.ack_nec.get()
        if ackNecStatus == 1:
            self.opc.write(('output.ACK_REQ', True))
            self.setLED('ACK_REQ')
        else:
            self.opc.write(('output.ACK_REQ', False))
            self.resetLED('ACK_REQ')
    
    def eStopPressed(self):
        self.updateAckReq()
        if self.buttonState == 1:
            self.buttonState = 0
            if(self.delay.get() == '0'): #category 1
                self.category = 0
                self.Q = 0
                self.opc.write(('output.Q', False))
                self.resetLED('Q')
                self.eStopButton['state'] = 'normal'
                self.eStopButton['text'] = 'Release'
                self.eStopButton['bg'] = 'green'
            else:
                self.eStopButton['state'] = 'disabled'
                self.category = 1
                self.after(int(self.delay.get()),lambda : self.delayedStop())  
                    
        elif self.buttonState == 0:
            if self.ack_nec.get() == 0:
                self.buttonState = 1
                self.opc.write(('output.Q', True))
                self.setLED('Q')
                self.opc.write(('output.Q_DELAY', True))
                self.setLED('Q_Delay')
                self.delay.set('0')
                self.eStopButton['text'] = 'E-STOP'
                self.eStopButton['bg'] = 'red'
            elif self.ack_nec.get() == 1 :
                if self.isRaisingEdge == 1 :
                    self.buttonState = 1
                    self.opc.write(('output.Q', True))
                    self.setLED('Q')
                    self.opc.write(('output.Q_DELAY', True))
                    self.setLED('Q_Delay')
                    self.delay.set('0')
                    self.eStopButton['text'] = 'E-STOP'
                    self.eStopButton['bg'] = 'red'
                    self.isRaisingEdge = 0

    def detectRaisingEdge(self):
        self.updateAckReq()
        current = E.ack.get()
        if current == 1 and E.ackPrevious == 0:
            E.isRaisingEdge = 1
        E.ackPrevious = current
        self.after(500,lambda : self.detectRaisingEdge())  

    def run(self):    
        self.after(500, lambda : self.detectRaisingEdge())
        self.mainloop()
        
    def __del__(self): 
        self.opc.close()       

In [42]:
E = EStop()
E.run()